In [1]:
import os
from dotenv import load_dotenv

from langchain_core.tools import tool, InjectedToolArg
from typing import Annotated

from langchain_openai import ChatOpenAI

from langchain.messages import HumanMessage

import json

import requests

In [2]:
load_dotenv()

True

In [3]:
CURRENCY_CONVERSION_API_KEY = os.getenv("CURRENCY_CONVERSION_API_KEY")
OPENAI_API_KEY = os.getenv("OPEN_AI_KEY")

In [4]:
# creating LLM
llm = ChatOpenAI(
    api_key = OPENAI_API_KEY,
    model="gpt-3.5-turbo",
    base_url="https://openrouter.ai/api/v1",
    verbose=True,
)

In [5]:
# tool creation - 1

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """
    This function fetches the currency conversion factor between a given base currency and a target currency
    """

    url = f"https://api.fastforex.io/fetch-one?from={base_currency}&to={target_currency}&api_key={CURRENCY_CONVERSION_API_KEY}"

    response = requests.get(url)

    return response.json()

    

In [6]:
# tool creation - 2

@tool
def multiply(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    # `InjectedToolArg` helps as LLM do Not try to fill this argument., I (the devloper/runtime) will inject this value after running earlier tools.
    """
    Given a currency conversion rate this function calculates the target currency value from a given a given base currency value
    """

    return base_currency_value * conversion_rate
    

In [7]:
get_conversion_factor.invoke({"base_currency": "USD", "target_currency":"INR"})

{'base': 'USD',
 'result': {'INR': 89.8672},
 'updated': '2025-12-26T13:19:51Z',
 'ms': 7}

In [8]:
multiply.invoke({"base_currency_value": 3, "conversion_rate": 89.9636})

269.8908

In [9]:
# tool binding

In [10]:
llm_with_tools = llm.bind_tools([get_conversion_factor, multiply])

In [11]:
llm_with_tools

RunnableBinding(bound=ChatOpenAI(profile={'max_input_tokens': 16385, 'max_output_tokens': 4096, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': False, 'structured_output': False, 'image_url_inputs': False, 'pdf_inputs': False, 'pdf_tool_message': False, 'image_tool_message': False, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x000002545E6160D0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002545ECBBC10>, root_client=<openai.OpenAI object at 0x000002545E614C90>, root_async_client=<openai.AsyncOpenAI object at 0x000002545ECBB950>, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://openrouter.ai/api/v1'), kwargs={'tools': [{'type': 'function', 'function': {'name': 'get_conversion_factor', 'description': 'This function fetche

In [12]:
query = HumanMessage("What is the conversion factor between USD and INR, and based on that can you convert 10 USD into INR")

In [13]:
messages = [query]

In [14]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that can you convert 10 USD into INR', additional_kwargs={}, response_metadata={})]

In [15]:
ai_message = llm_with_tools.invoke(messages)

In [16]:
messages.append(ai_message)

In [17]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that can you convert 10 USD into INR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 124, 'total_tokens': 176, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None, 'image_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0, 'video_tokens': 0}, 'cost': 0.00014, 'is_byok': False, 'cost_details': {'upstream_inference_cost': None, 'upstream_inference_prompt_cost': 6.2e-05, 'upstream_inference_completions_cost': 7.8e-05}}, 'model_provider': 'openai', 'model_name': 'openai/gpt-3.5-turbo', 'system_fingerprint': None, 'id': 'gen-1766755221-q4QsRdvkpYickZQZclsR', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b5ad1-3995-76a2-8a0d-deaaa953056

In [18]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'call_aAtKEdpYIXVtkOjgrn4adFD9',
  'type': 'tool_call'},
 {'name': 'multiply',
  'args': {'base_currency_value': 10},
  'id': 'call_VfcKuLQacAFaN4ZE07tY0sUO',
  'type': 'tool_call'}]

In [19]:
# tool execution
for tool_call in ai_message.tool_calls:

    # execute the 1st tool and get the value of conversion rate
    if tool_call['name'] == "get_conversion_factor":
        tool_message1 = get_conversion_factor.invoke(tool_call)
        
        # fetch this conversion rate
        temp = json.loads(tool_message1.content)
        conversion_rate = list(temp['result'].values())[0]
    
        # append this tool message to messages list
        messages.append(tool_message1)
    

    # execute the 2nd tool using the conversion rate form tool 1
    if tool_call['name'] == 'multiply':
        tool_call['args']['conversion_rate'] = conversion_rate

        tool_message2 = multiply.invoke(tool_call)

        # append this tool message to messages list
        messages.append(tool_message2)



In [25]:
type(tool_message1)

langchain_core.messages.tool.ToolMessage

In [20]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that can you convert 10 USD into INR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 124, 'total_tokens': 176, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None, 'image_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0, 'video_tokens': 0}, 'cost': 0.00014, 'is_byok': False, 'cost_details': {'upstream_inference_cost': None, 'upstream_inference_prompt_cost': 6.2e-05, 'upstream_inference_completions_cost': 7.8e-05}}, 'model_provider': 'openai', 'model_name': 'openai/gpt-3.5-turbo', 'system_fingerprint': None, 'id': 'gen-1766755221-q4QsRdvkpYickZQZclsR', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b5ad1-3995-76a2-8a0d-deaaa953056

In [32]:
final_result = llm.invoke(messages)

In [34]:
final_result.content

'The conversion factor between USD and INR is 1 USD = 89.8672 INR. Therefore, 10 USD is equivalent to 898.67 INR.'